In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
from scrapy.selector import Selector
from scrapy.http import HtmlResponse


In [37]:
def get_log_data(gamelogs_url, year):
    
    #url = 'http://www.baseball-reference.com/players/gl.cgi?id=crawfbr01&t=b&year=2015'
    url = gamelogs_url
    #print url
    r = requests.get(url)
    #    print "got to site"
    b = BeautifulSoup(r.text, 'html.parser')
    HTML = r.text 

    #Initialize row identifier
    RK = 1
    end_of_table = False
    my_log_data = {}
    
    #Declare a new instance of a Dataframe log
    log_data = pd.DataFrame(columns=(
        'RK', 'Gcar', 'Gtm', 'Date', 'Tm', 'HoA', 'Opp', 'Rslt',
       'Inngs', 'PA', 'AB', 'R', 'H', 'Doubles', 'Tripples', 'HR',
       'RBI', 'BB', 'IBB', 'SO', 'HBP', 'SH', 'SF', 'ROE', 'GDP',
       'SB', 'CS', 'BA', 'OBP', 'SLG', 'OPS', 'BOP', 'aLI', 'WPA',
       'RE24', 'DK', 'FD', 'Pos')
    )
    
    #//thead/tr/th
    columns = [''.join(td.xpath('.//text()').extract()) for td in Selector(text=HTML).xpath('//thead/tr/th')]
    print(columns)
    print(log_data.columns)
    
    #While RK is not null (i.e at the end of the table...)
    #increment RK and collect pitching totals
    while(end_of_table == False):

        #Gate Keeping code to determine if while loop should proceed
        try: 
            #This is where the table is found: //div[3]/div[2]/div/table/tbody/tr[1]/td
            fields = [''.join(td.xpath('.//text()').extract()) for td in Selector(text=HTML).xpath('//div[3]/div[2]/div/table/tbody/tr['+str(RK)+']/td')]
            print("There are {} fields. Here they are: {}".format(len(fields), fields))
            ahrefs = Selector(text=HTML).xpath('//div[3]/div[2]/div/table/tbody/tr['+str(RK)+']/td/a').extract()
            row_number = fields[0].encode('utf-8').strip()

        except:
            try:
                #Is this because we have encoutered a "title" row? As opposed to end of table?
                fields = [''.join(td.xpath('.//text()').extract()) for td in Selector(text=HTML).xpath('//div[3]/div[2]/div/table/tbody/tr['+str(RK+1)+']/td')]
                #fields = Selector(text=HTML).xpath('//div[3]/table/tbody/tr['+str(RK+1)+']/td/text()').extract()
                ahrefs = Selector(text=HTML).xpath('//div[3]/div[2]/div/table/tbody/tr['+str(RK+1)+']/td/a').extract()
                row_number = fields[0].encode('utf-8').strip()
                print(fields)
                
                RK += 1
            except:
                #We have reached the end of the table
                end_of_table = True
                print(year, RK)

        #print year, row_number, len(fields), len(ahrefs)
        if len(fields) == 37:     #Table has DK, and FD information
            print('37')
   
            if(row_number not in my_log_data.keys()):
                
                my_log_data[row_number] = {}
                
            if(fields[0] not in my_log_data[row_number].keys()):
                my_log_data[row_number][columns[0]] = fields[0]
                
            end_of_table=True

#             log_data.loc[row_number, "Year"] = year
#             log_data.loc[row_number, "RK"] = row_number
#             log_data.loc[row_number, "Gcar"] = fields[1]
#             log_data.loc[row_number, "Gtm"] = fields[2].encode('utf-8').strip()
#             log_data.loc[row_number, "Date"] = fields[3].encode('utf-8').strip()
#             log_data.loc[row_number, "Tm"] = fields[4].encode('utf-8').strip()
#             log_data.loc[row_number, "HoA"] = fields[5].encode('utf-8').strip()
#             log_data.loc[row_number, "Opp"] = fields[6].encode('utf-8').strip()
#             log_data.loc[row_number, "Rsit"] = fields[7].encode('utf-8').strip()
#             log_data.loc[row_number, "Inngs"] = fields[8].encode('utf-8').strip()
#             log_data.loc[row_number, "PA"] = fields[9].encode('utf-8').strip()
#             log_data.loc[row_number, "AB"] = fields[10].encode('utf-8').strip()
#             log_data.loc[row_number, "R"] = fields[11].encode('utf-8').strip()
#             log_data.loc[row_number, "H"] = fields[12].encode('utf-8').strip()
#             log_data.loc[row_number, "Doubles"] = fields[13].encode('utf-8').strip()
#             log_data.loc[row_number, "Tripples"] = fields[14].encode('utf-8').strip()
#             log_data.loc[row_number, "HR"] = fields[15].encode('utf-8').strip()
#             log_data.loc[row_number, "RBI"] = fields[16].encode('utf-8').strip()
#             log_data.loc[row_number, "BB"] = fields[17].encode('utf-8').strip()
#             log_data.loc[row_number, "IBB"] = fields[18].encode('utf-8').strip()
#             log_data.loc[row_number, "SO"] = fields[19].encode('utf-8').strip()
#             log_data.loc[row_number, "HBP"] = fields[20].encode('utf-8').strip()
#             log_data.loc[row_number, "SH"] = fields[21].encode('utf-8').strip()
#             log_data.loc[row_number, "SF"] = fields[22].encode('utf-8').strip()
#             log_data.loc[row_number, "ROE"] = fields[23].encode('utf-8').strip()
#             log_data.loc[row_number, "GDP"] = fields[24].encode('utf-8').strip()
#             log_data.loc[row_number, "SB"] = fields[25].encode('utf-8').strip()
#             log_data.loc[row_number, "CS"] = fields[26].encode('utf-8').strip()
#             log_data.loc[row_number, "BA"] = fields[27].encode('utf-8').strip()
#             log_data.loc[row_number, "OBP"] = fields[28].encode('utf-8').strip()
#             log_data.loc[row_number, "SLG"] = fields[29].encode('utf-8').strip()
#             log_data.loc[row_number, "OPS"] = fields[30].encode('utf-8').strip()
#             log_data.loc[row_number, "BOP"] = fields[31].encode('utf-8').strip()
#             log_data.loc[row_number, "aLI"] = fields[32].encode('utf-8').strip()
#             log_data.loc[row_number, "WPA"] = fields[33].encode('utf-8').strip()
#             log_data.loc[row_number, "RE24"] = fields[34].encode('utf-8').strip()
#             log_data.loc[row_number, "DK"] = fields[35].encode('utf-8').strip()
#             log_data.loc[row_number, "FD"] = fields[36].encode('utf-8').strip()
#             log_data.loc[row_number, "Pos"] = fields[37].encode('utf-8').strip()
            
        elif len(fields) == 36:
            print('36')
            log_data.loc[row_number, "Year"] = year
            log_data.loc[row_number, "RK"] = row_number
            log_data.loc[row_number, "Gcar"] = fields[1].encode('utf-8').strip()
            log_data.loc[row_number, "Gtm"] = fields[2].encode('utf-8').strip()
            log_data.loc[row_number, "Date"] = fields[3].encode('utf-8').strip()
            log_data.loc[row_number, "Tm"] = fields[4].encode('utf-8').strip()
            log_data.loc[row_number, "HoA"] = fields[5].encode('utf-8').strip()
            log_data.loc[row_number, "Opp"] = fields[6].encode('utf-8').strip()
            log_data.loc[row_number, "Rsit"] = fields[7].encode('utf-8').strip()
            log_data.loc[row_number, "Inngs"] = fields[8].encode('utf-8').strip()
            log_data.loc[row_number, "PA"] = fields[9].encode('utf-8').strip()
            log_data.loc[row_number, "AB"] = fields[10].encode('utf-8').strip()
            log_data.loc[row_number, "R"] = fields[11].encode('utf-8').strip()
            log_data.loc[row_number, "H"] = fields[12].encode('utf-8').strip()
            log_data.loc[row_number, "Doubles"] = fields[13].encode('utf-8').strip()
            log_data.loc[row_number, "Tripples"] = fields[14].encode('utf-8').strip()
            log_data.loc[row_number, "HR"] = fields[15].encode('utf-8').strip()
            log_data.loc[row_number, "RBI"] = fields[16].encode('utf-8').strip()
            log_data.loc[row_number, "BB"] = fields[17].encode('utf-8').strip()
            log_data.loc[row_number, "IBB"] = fields[18].encode('utf-8').strip()
            log_data.loc[row_number, "SO"] = fields[19].encode('utf-8').strip()
            log_data.loc[row_number, "HBP"] = fields[20].encode('utf-8').strip()
            log_data.loc[row_number, "SH"] = fields[21].encode('utf-8').strip()
            log_data.loc[row_number, "SF"] = fields[22].encode('utf-8').strip()
            log_data.loc[row_number, "ROE"] = fields[23].encode('utf-8').strip()
            log_data.loc[row_number, "GDP"] = fields[24].encode('utf-8').strip()
            log_data.loc[row_number, "SB"] = fields[25].encode('utf-8').strip()
            log_data.loc[row_number, "CS"] = fields[26].encode('utf-8').strip()
            log_data.loc[row_number, "BA"] = fields[27].encode('utf-8').strip()
            log_data.loc[row_number, "OBP"] = fields[28].encode('utf-8').strip()
            log_data.loc[row_number, "SLG"] = fields[29].encode('utf-8').strip()
            log_data.loc[row_number, "OPS"] = fields[30].encode('utf-8').strip()
            log_data.loc[row_number, "BOP"] = fields[31].encode('utf-8').strip()
            log_data.loc[row_number, "aLI"] = fields[32].encode('utf-8').strip()
            log_data.loc[row_number, "WPA"] = fields[33].encode('utf-8').strip()
            log_data.loc[row_number, "RE24"] = fields[34].encode('utf-8').strip()
            log_data.loc[row_number, "DK"] = ""
            log_data.loc[row_number, "FD"] = ""
            log_data.loc[row_number, "Pos"] = fields[35].encode('utf-8').strip()
        else:
            exception_cases.append(row_number)
        
        RK += 1

#         if RK > 3:
#             end_of_table = True
    
    return my_log_data


In [10]:
#Example for how to get game logs for a single player
url = 'http://www.baseball-reference.com/players/gl.cgi?id=crawfbr01&t=b&year=2015'
r = requests.get(url)
#    print "got to site"
b = BeautifulSoup(r.text, 'html.parser')
HTML = r.text 


#xpath for Gamelogs:  //li[5]/div/ul[1]/li/a
fields = Selector(text=HTML).xpath('//li[5]/div/ul[1]/li/a').extract()
print(fields)
print(len(fields))
extracted_url = fields[len(fields)-2]

try:
    found = re.search('<a href="(.+?)">', extracted_url).group(1)
except AttributeError:
    found = '' # apply some error handling
#found

#Concatenate the strings that will be used as full url to follow for data
gamelogs_url = "http://www.baseball-reference.com" + found

print(gamelogs_url)

['<a href="/players/gl.fcgi?id=crawfbr01&amp;t=b&amp;year=2011">2011</a>', '<a href="/players/gl.fcgi?id=crawfbr01&amp;t=b&amp;year=2012">2012</a>', '<a href="/players/gl.fcgi?id=crawfbr01&amp;t=b&amp;year=2013">2013</a>', '<a href="/players/gl.fcgi?id=crawfbr01&amp;t=b&amp;year=2014">2014</a>', '<a href="/players/gl.fcgi?id=crawfbr01&amp;t=b&amp;year=2015">2015</a>', '<a href="/players/gl.fcgi?id=crawfbr01&amp;t=b&amp;year=2016">2016</a>', '<a href="/players/gl.fcgi?id=crawfbr01&amp;t=b&amp;year=2017">2017</a>', '<a href="/players/gl.fcgi?id=crawfbr01&amp;t=b&amp;year=0&amp;post=1">Postseason</a>']
8
http://www.baseball-reference.com/players/gl.fcgi?id=crawfbr01&amp;t=b&amp;year=2017


In [40]:
#Concatenate the strings that will be used as full url to follow for data
#gamelogs_url = "http://www.baseball-reference.com" + found
gamelogs_url = "http://www.baseball-reference.com/players/gl.fcgi?id=crawfbr01&amp;t=b&amp;year=2017"

#years_of_interest = np.arange(2011, 2017, 1)
years_of_interest = [2017]
years_of_interest = sorted(years_of_interest, reverse=True)
#years_of_interest

batter_log = pd.DataFrame()
buffer_df = pd.DataFrame()


for year in years_of_interest:
    url = gamelogs_url
    #print url
    try:
        my_dict = get_log_data(url,year)
        #buffer_df = get_log_data(url, year)
        #batter_log = batter_log.append(buffer_df)

    except:
        #We have found a year log that does not exist for this player and it is likely that the preceeding years
        #also do not exist...
        break
    
#opponent = get_opposing_pitch_totals(url)

print(my_dict)


['Rk', 'Gcar', 'Gtm', 'Date', 'Tm', '', 'Opp', 'Rslt', 'Inngs', 'PA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB', 'IBB', 'SO', 'HBP', 'SH', 'SF', 'ROE', 'GDP', 'SB', 'CS', 'BA', 'OBP', 'SLG', 'OPS', 'BOP', 'aLI', 'WPA', 'RE24', 'DFS(DK)', 'DFS(FD)', 'Pos']
Index(['RK', 'Gcar', 'Gtm', 'Date', 'Tm', 'HoA', 'Opp', 'Rslt', 'Inngs', 'PA',
       'AB', 'R', 'H', 'Doubles', 'Tripples', 'HR', 'RBI', 'BB', 'IBB', 'SO',
       'HBP', 'SH', 'SF', 'ROE', 'GDP', 'SB', 'CS', 'BA', 'OBP', 'SLG', 'OPS',
       'BOP', 'aLI', 'WPA', 'RE24', 'DK', 'FD', 'Pos'],
      dtype='object')
There are 37 fields. Here they are: ['810', '1', 'Apr 2', 'SFG', '@', 'ARI', 'L,5-6', 'CG', '5', '5', '1', '2', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '.400', '.400', '.600', '1.000', '5', '1.31', '-0.055', '-0.73', '', '', 'SS']
37
{b'810': {'Rk': '810'}}
